In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from catboost import Pool, CatBoostRegressor
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler



In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
sample_submission = pd.read_csv("../input/house-prices-advanced-regression-techniques/sample_submission.csv")
test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train.dtypes.tolist()

In [ ]:
train.describe()

In [ ]:
train_Var = train[train.dtypes[train.dtypes == "int64"].index]

train_Cat = train[train.dtypes[train.dtypes == "object"].index]


In [ ]:
train_Var.head()


In [ ]:
train_Var.isna().sum().max()

In [ ]:
train_Cat.head()

In [ ]:
train_Cat.isna().sum().max()

In [ ]:
#checking if the dataframe contains null
def missing_total(data):
    missing_total= data.isna().sum().sort_values(ascending=False)
    return missing_total
#missing_total= train.isna().sum().sort_values(ascending=False)

In [ ]:
def missing_percent(data):
    missing_percent = ((data.isna().sum()/data.isna().count())*100).sort_values(ascending=False)
    return missing_percent


In [ ]:
train_missing = pd.concat([missing_total(train_Cat), missing_percent(train_Cat)], axis=1, keys=['missing_total', 'missing_percent'])
train_missing.head(50)

In [ ]:
#further anaylsis on missing train data
train_Cat = train_Cat.drop((train_missing[train_missing['missing_percent'] > 10.0]).index,1)


In [ ]:
train_Cat.head(5)

In [ ]:
#new_train_var = train_Var.drop(['BsmtFinSF2','PoolArea','MiscVal','ScreenPorch','3SsnPorch','EnclosedPorch','OpenPorchSF','WoodDeckSF','2ndFlrSF','LowQualFinSF','PoolArea','MiscVal','Id'], axis=1)

In [ ]:
#new_train_var.head(5)

In [ ]:
train_Cat = pd.get_dummies(train_Cat)

In [ ]:
#replace every NaN with the first non-NaN value in the same column above it
#train_Cat.fillna(method='median', inplace=True)
train_Cat.fillna(train_Cat.median(), inplace=True)
#imputer = SimpleImputer(missing_values=np.nan, strategy='median')
#imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
#train_Cat = imputer.fit_transform(train_Cat)

In [ ]:
train_Cat

In [ ]:
inputData=train_Var.join(train_Cat)

In [ ]:
inputData.head(5)

In [ ]:
inputData.isna().sum().max()

In [ ]:
inputData = inputData.drop(['Id'],axis=1)

Final Prediction Data Pre-processing

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
test.columns

In [ ]:
test_Var = test[test.dtypes[test.dtypes == "int64"].index]

test_Cat = test[test.dtypes[test.dtypes == "object"].index]

In [ ]:
test_Var.isna().sum().max() 

In [ ]:
test_Cat.isna().sum().max() 

In [ ]:
test_missing = pd.concat([missing_total(test_Cat), missing_percent(test_Cat)], axis=1, keys=['missing_total', 'missing_percent'])
test_missing.head(50)

In [ ]:
#further anaylsis on missing train data
test_Cat = test_Cat.drop((test_missing[test_missing['missing_percent'] > 10.0]).index,1)

In [ ]:
test_Cat.head(5)

In [ ]:
test_Cat.isna().sum().max()

In [ ]:
#new_test_var = test_Var.drop(['PoolArea','MiscVal','ScreenPorch','3SsnPorch','EnclosedPorch','OpenPorchSF','WoodDeckSF','2ndFlrSF','LowQualFinSF','PoolArea','MiscVal','Id'], axis=1)

In [ ]:
#new_test_var.head(5)

In [ ]:
test_Cat = pd.get_dummies(test_Cat)

In [ ]:
#replace every NaN with the first non-NaN value in the same column above it
test_Cat.fillna(test_Cat.mean(), inplace=True)

In [ ]:
outputData=test_Var.join(test_Cat)

In [ ]:
outputData =outputData.drop(['Id'],axis=1)

In [ ]:
diff_data = inputData.columns.difference(outputData.columns)

In [ ]:
diff_data

In [ ]:
inputData = inputData.drop( ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn',
       'Electrical_Mix', 'Exterior1st_ImStucc', 'Exterior1st_Stone',
       'Exterior2nd_Other', 'GarageArea', 'GarageCars', 'GarageQual_Ex',
       'Heating_Floor', 'Heating_OthW', 'HouseStyle_2.5Fin',
       'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Metal',
       'RoofMatl_Roll',  'TotalBsmtSF', 'Utilities_NoSeWa'],axis=1)

In [ ]:
diff_data2 = outputData.columns.difference(inputData.columns)

In [ ]:
diff_data2

In [ ]:
print(inputData.shape, outputData.shape)

In [ ]:
outputData.head(5)

In [ ]:
inputData.head(5)

Data Splitting and Scalling

In [ ]:
X_train = inputData.drop(['SalePrice'],axis=1)#.values
Y_train = inputData['SalePrice'].values
X_final_pred =outputData.values

In [ ]:
# Log transformation
#X_train = np.log(X_train)
#Y_train = np.log(Y_train)
#X_final_pred = np.log(X_final_pred)

In [ ]:
# Change -inf to 0 again
#X_train[X_train==-np.inf]=0
#X_final_pred[X_final_pred==-np.inf]=0

In [ ]:
X_train

In [ ]:
seed = 400
np.random.seed(seed)
X_training, X_testing, Y_training, Y_testing= train_test_split(X_train, Y_train.reshape(-1,1), test_size=0.1, random_state=seed)
X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()

X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)
X_final_pred_scaled = X_scaler.transform(X_final_pred)

In [ ]:
X_scaled_training.shape

In [ ]:
X_final_pred_scaled.shape

In [ ]:
features_selection = LassoCV(0.05, cv=10)
reg = SelectFromModel(features_selection)
reg.fit(X_scaled_training, Y_scaled_training)

predictedFeatures = reg.transform(X_scaled_training)
#jack=X_scaled_training[reg.get_support(indices=True)]

#Printing the features selected
for feature_list_index in reg.get_support(indices=True):
    print([feature_list_index])

In [ ]:
#Transforming the testing data to the actual number of features selected for the training data
testingFeatures = reg.transform(X_scaled_testing)

In [ ]:
X_final_pred_scaled = reg.transform(X_final_pred_scaled )

In [ ]:
#'DIMENTIONAL REDUCTION'
pca = PCA(0.95,whiten=True)

pca_X_scaled_training = pca.fit_transform(predictedFeatures)

# To get how many principal components was chosen
print(pca.n_components_)
explained_variance=pca.explained_variance_ratio_
print(explained_variance)
print(pca.explained_variance_ratio_.sum())
#Plot of distribution of the principal component
with plt.style.context('dark_background'):
    plt.figure(figsize=(60, 40))

    plt.bar(range(len(pca_X_scaled_training[0])), pca.explained_variance_ratio_, alpha=0.5, align='center',
            label='individual explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()

In [ ]:
X_scaled_training =pca_X_scaled_training

In [ ]:
X_scaled_testing = pca.transform(testingFeatures)

In [ ]:
X_final_pred_scaled = pca.transform(X_final_pred_scaled )

Model Development

In [ ]:
#definition of model for prediction
# initialize Pool
train_pool = Pool(X_scaled_training, 
                  Y_scaled_training)
test_pool = Pool(X_scaled_testing, 
                 Y_scaled_testing) 





In [ ]:
model = CatBoostRegressor(iterations=200, 
                          depth=5, 
                          learning_rate=0.09, 
                          loss_function='MAE')
model.fit(train_pool)

In [ ]:
preds = model.predict(test_pool)

In [ ]:
r2= r2_score(Y_scaled_testing, preds)
mae= mean_absolute_error(Y_scaled_testing, preds)
mse= mean_squared_error(Y_scaled_testing, preds)

In [ ]:
print(r2, mae, mse)

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.layers import ReLU, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD


In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout,Input
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

In [ ]:
def keras_prediction():
# create model
    model = Sequential()
    #input layer
    model.add(Dense(35, input_shape=(X_scaled_training.shape[1],),activation='linear'))
    #model.add(Dropout(0.1))
    #model.add(BatchNormalization())
    #hidden layers
    model.add(tf.keras.layers.Dense(10, activation='linear'))
    #model.add(BatchNormalization())
    #model.add(Dense(10, activation='relu'))
    #model.add(BatchNormalization())
    #output layer
    model.add(Dense(1,activation='linear'))
    optimizer = Adam(lr=0.05)
# Compile model
    model.compile(loss='mae', optimizer='adam',metrics=[tf.keras.metrics.RootMeanSquaredError(),coeff_determination])
    return model

In [ ]:
#Custom function for coefficient of determination
def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon())) #To avoid division by zero, I use K.epsilon() which has the value 1E-8.

In [ ]:
early_stopping_monitor = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, mode='min')

In [ ]:
model = keras_prediction()

In [ ]:
history= model.fit(X_scaled_training,Y_scaled_training,validation_split=0.2,epochs=100, batch_size= 74,verbose=1,callbacks=[early_stopping_monitor])

In [ ]:
preds2 = model.predict(X_scaled_testing)

In [ ]:
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.show()

In [ ]:
r22= r2_score(Y_scaled_testing, preds2)
mae2= mean_absolute_error(Y_scaled_testing, preds2)
mse2= mean_squared_error(Y_scaled_testing, preds2)

In [ ]:
print(r22, mae2, mse2)

Prediction for submission

In [ ]:
sample_submission.head()

In [ ]:
yhat = model.predict(X_final_pred_scaled)

In [ ]:
prediction = Y_scaler.inverse_transform(yhat)

In [ ]:
Prediction = pd.concat([pd.DataFrame(test_Var['Id'], columns=['Id']),pd.DataFrame(prediction, columns=['SalePrice'])],axis=1)

In [ ]:
Prediction

In [ ]:
Prediction.to_csv("submisssion6.csv", index=False)